In [1]:
import numpy as np
import alienlab
import pandas as pd
import matplotlib.pyplot as plt

from mvgavg import mvgavg
from tkinter.filedialog import askopenfilename
import os

from   scipy import optimize

%matplotlib ipympl

# Bode Diagram

**Experiment description:**   
No intensity filter on LEDs  
Filter x1e-2 on MPPC fluo  

480 constant 4V  
405 modulated offset 2V, amplitude 2V  

frequency range : 
    N = 30
    frequencies = 10**np.linspace(-2, 3, N)

### Log

see archive file

In [15]:
from statsmodels.regression import linear_model
from statsmodels.api import add_constant
from scipy.interpolate import InterpolatedUnivariateSpline
import numpy as np





def get_func(X, Y):
        Y = np.array([y for x, y in sorted(zip(X, Y))]) #preliminary sorting of the arrays along wavelength 
                                                            #(in case the graph in not properly ordered)
        X = np.sort(X)
        func = InterpolatedUnivariateSpline(X, Y) # interpolate given values with step 1 nm
        return func

def regression_affine(X, Y, details = True):
        Xreg = add_constant(X) #Add a constant to fit tan affine model

        model = linear_model.OLS(Y, Xreg) #Linear regression
        results = model.fit()
        [b, a] = results.params #parameters of the affine curve
        Yreg = a*X + b #regression curve

        return Yreg, a, b, results.summary()

def exp_decay(parameters,xdata):
    '''
    Calculate an exponetial decay of the form:
    S= a * exp(-xdata/b)
    '''
    A = parameters[0]
    tau = parameters[1]
    y0 = parameters[2]
    return A * np.exp(-xdata/tau) + y0

def band_pass(parameters, xdata):
    
    H = parameters[0]
    tau = parameters[1]
    a0 = parameters[2]
    
    return H * (xdata * tau) /(1 + (xdata  * tau)**2) + a0

def low_pass(parameters, xdata):
    
    H = parameters[0]
    tau = parameters[1]
    a0 = parameters[2]
    
    return H*(xdata * tau)**2 /(1 + (xdata * tau)**2) + a0

def high_pass(parameters, xdata):
    
    H = parameters[0]
    tau = parameters[1]
    a0 = parameters[2]
    
    return H /(1 + (xdata * tau)**2) + a0



def amplitude(parameters, xdata):
    
    H = parameters[0]
    tau = parameters[1]
    a0 = parameters[2]
    
    return (H /(1 + (xdata * tau)**2))**0.5 + a0

def residuals(parameters,x_data,y_observed,func):
    '''
    Compute residuals of y_predicted - y_observed
    where:
    y_predicted = func(parameters,x_data)
    '''
    return func(parameters,x_data) - y_observed


def averaging(measured_signal, 
                time_array, window, 
                do_binning = True):

    average_output = mvgavg(measured_signal, window, axis = 1, binning = do_binning)
    downscaled_time = time_array[window//2 :: window] 
    #todo: if binning == False, time array
    return average_output, downscaled_time    

def lock_in(self, measured_signal, 
            time_array, 
            frequency, phase_shift):
    cos_ref = np.cos(2*np.pi * frequency * time_array - phase_shift)
    cos_ref = np.stack([cos_ref] * measured_signal.shape[0])
    sin_ref = np.sin(2*np.pi * frequency * time_array - phase_shift)
    sin_ref = np.stack([sin_ref] * measured_signal.shape[0])
    cos_lock = 2 * np.multiply(measured_signal, cos_ref)
    sin_lock =  2 * np.multiply(measured_signal, sin_ref)
    radius_lock = np.sqrt(sin_lock.mean(axis = 1)**2 + cos_lock.mean(axis = 1)**2)
    phase_lock = np.arctan(sin_lock.mean(axis = 1)/cos_lock.mean(axis = 1))
    return sin_lock, cos_lock, radius_lock, phase_lock

def FFT(t, y):
    #source: https://stackoverflow.com/questions/56797881/how-to-properly-scale-frequency-axis-in-fast-fourier-transform
    n = len(t)
    Δ = (max(t) - min(t)) / (n-1)
    k = int(n/2)
    f = np.arange(k) / (n*Δ)
    Y = np.abs(np.fft.fft(y))[:k]
    return (f, Y)


In [3]:
file = "bode_full_response.csv"
curve = pd.read_csv(file, names = ['intensity', 'fluo', 'tick', 'trigger', 'd', 'e', 'f'], sep = ',', decimal = '.') 

intensity = curve.intensity.values
fluo = curve.fluo.values
tick = curve.tick.values


In [4]:
plt.figure(figsize = (20, 4))
plt.plot(fluo)
plt.title('Raw fluorescence signal, 10 period per frequency')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
time = np.linspace(0, len(fluo)-1, len(fluo))
f = 1 / (10000)
sinus = np.sin(time * f*2*np.pi)
cosinus = np.cos(time * f * 2 * np.pi)

In [6]:

window = 200
both = np.stack([fluo, tick], axis = 1)
smooth, time_bis = averaging(both.T, time, window)

In [7]:
plt.figure(figsize = (20, 4))
plt.plot(smooth[0])
plt.title('smoothed fluorescence signal, 10 period per frequency')
#plt.plot(smooth[1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'smoothed fluorescence signal, 10 period per frequency')

In [8]:
plt.figure(figsize = (20, 4))
plt.plot(tick)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
import matplotlib
cmap = matplotlib.cm.get_cmap('Spectral')
color_list = [cmap(x) for x in np.linspace(0, 10, 10)]
rgba = cmap(0.5)
all_outputs_3D = np.load("bode_3D_output.npy")
all_times = np.load("bode_3D_times.npy")
print(all_outputs_3D.shape)
fig, axs =  plt.subplots(2, 2, figsize=(15, 6))
for i in range(all_outputs_3D.shape[0]):
    f, Y = FFT(all_times[0], all_outputs_3D[i][1])
    axs[1][0].plot(f[:100], Y[:100])
    axs[1][0].set_title("Fourier transform fluorescence")
    Y = Y[:100]
    axs[1][1].plot(np.linspace(0, len(Y)//10, len(Y)//10), Y[::10]/Y[10], c = color_list[i])
    axs[1][1].set_title("Harmonics fluorescence")
    
    f, Y = FFT(all_times[0], all_outputs_3D[i][0])
    axs[0][0].plot(f[:100], Y[:100])
    axs[0][0].set_title("Fourier transform input")
    Y = Y[:100]
    axs[0][1].plot(np.linspace(0, len(Y)//10, len(Y)//10), Y[::10]/Y[10], c = color_list[i])
    axs[0][1].set_title("Harmonics input")


(10, 7, 20000)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [53]:
plt.figure()
plt.plot(mvgavg(all_outputs_3D[5][2], 100))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
files = os.listdir()
ind = [i for i, elem in enumerate(files) if 'fluo.csv' in elem]
path =files[ind[0]]


In [10]:
curve = pd.read_csv(path, sep = ',', decimal = '.') 

K = curve.keys()
d = 0
f = -5
frequencies = curve[K[1]].values[d:f]
radius = curve[K[2]].values[d:f]
sin_lo = curve[K[4]].values[d:f]
cos_lo = curve[K[6]].values[d:f]
phase = curve[K[8]].values[d:f]

l = 3
radius[0:l] = radius[l + 1]
sin_lo[0:l] = sin_lo[l + 1]
cos_lo[0:l] = cos_lo[l + 1]


In [11]:

x0 = [0.1, 0.1, 0]

pulse = 2 * np.pi * frequencies
parameters_estimated = optimize.least_squares(residuals,  x0, bounds = (-1e5,1e5),
                                    args = (pulse, sin_lo, band_pass)).x
tau_sin = parameters_estimated[1]
print("cut-off cos: ", tau_sin)
plt.figure()
plt.plot(pulse, band_pass(parameters_estimated, pulse))
plt.plot(pulse, sin_lo)
plt.xscale('log')
plt.title('Cos lock-in')
plt.figure()
parameters_estimated = optimize.least_squares(residuals,  x0, bounds = (-1e5,1e5),
                                    args = (pulse, cos_lo, low_pass)).x
tau_cos = parameters_estimated[1]
print("cut-off cos: ", tau_sin)

plt.plot(pulse, low_pass(parameters_estimated, pulse))
plt.plot(pulse, cos_lo)
plt.xscale('log')
plt.title('Sin lock-in')

plt.figure()
parameters_estimated = optimize.least_squares(residuals,  x0, bounds = (-1e5,1e5),
                                    args = (pulse, radius, amplitude)).x
tau_radius = parameters_estimated[1]
print("cut-off radius:", tau_radius)
plt.plot(pulse, amplitude(parameters_estimated, pulse))
plt.plot(pulse, radius)
plt.xscale('log')
plt.title('Amplitude lock-in')

plt.show()


cut-off cos:  -0.001307419898285019


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

cut-off cos:  -0.001307419898285019


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

cut-off radius: 0.1912136784953551


<ipython-input-2-0c9e6ec0f2dc>:69: RuntimeWarning: invalid value encountered in sqrt
  return (H /(1 + (xdata * tau)**2))**0.5 + a0
